# Dynamic Recurrent Neural Network.

TensorFlow 2.0 implementation of a Recurrent Neural Network (LSTM) that performs dynamic computation over sequences with variable length. This example is using a toy dataset to classify linear sequences. The generated sequences have variable length.

In [6]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow v2.
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np
import random

In [7]:
# Dataset parameters.
num_classes = 2 # linear sequence or not.
seq_max_len = 20 # Maximum sequence length.
seq_min_len = 5 # Minimum sequence length (before padding).
masking_val = -1 # -1 will represents the mask and be used to pad sequences to a common max length.
max_value = 10000 # Maximum int value.

# Training Parameters
learning_rate = 0.001
training_steps = 2000
batch_size = 64
display_step = 100

# Network Parameters
num_units = 32 # number of neurons for the LSTM layer.

In [8]:
# ====================
#  TOY DATA GENERATOR
# ====================

def toy_sequence_data():
    """ Generate sequence of data with dynamic length.
    This function generates toy samples for training:
    - Class 0: linear sequences (i.e. [1, 2, 3, 4, ...])
    - Class 1: random sequences (i.e. [9, 3, 10, 7,...])

    NOTICE:
    We have to pad each sequence to reach 'seq_max_len' for TensorFlow
    consistency (we cannot feed a numpy array with inconsistent
    dimensions). The dynamic calculation will then be perform and ignore
    the masked value (here -1).
    """
    while True:
        # Set variable sequence length.
        seq_len = random.randint(seq_min_len, seq_max_len)
        rand_start = random.randint(0, max_value - seq_len)
        # Add a random or linear int sequence (50% prob).
        if random.random() < .5:
            # Generate a linear sequence.
            seq = np.arange(start=rand_start, stop=rand_start+seq_len)
            # Rescale values to [0., 1.].
            seq = seq / max_value
            # Pad sequence until the maximum length for dimension consistency.
            # Masking value: -1.
            seq = np.pad(seq, mode='constant', pad_width=(0, seq_max_len-seq_len), constant_values=masking_val)
            label = 0
        else:
            # Generate a random sequence.
            seq = np.random.randint(max_value, size=seq_len)
            # Rescale values to [0., 1.].
            seq = seq / max_value
            # Pad sequence until the maximum length for dimension consistency.
            # Masking value: -1.
            seq = np.pad(seq, mode='constant', pad_width=(0, seq_max_len-seq_len), constant_values=masking_val)
            label = 1
        yield np.array(seq, dtype=np.float32), np.array(label, dtype=np.float32)

In [9]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_generator(toy_sequence_data, output_types=(tf.float32, tf.float32))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [10]:
# Create LSTM Model.
class LSTM(Model):
    # Set layers.
    def __init__(self):
        super().__init__()
        # Define a Masking Layer with -1 as mask.
        self.masking = layers.Masking(mask_value=masking_val)
        # Define a LSTM layer to be applied over the Masking layer.
        # Dynamic computation will automatically be performed to ignore -1 values.
        self.lstm = layers.LSTM(units=num_units)
        # Output fully connected layer (2 classes: linear or random seq).
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        # A RNN Layer expects a 3-dim input (batch_size, seq_len, num_features).
        x = tf.reshape(x, shape=[-1, seq_max_len, 1])
        # Apply Masking layer.
        x = self.masking(x)
        # Apply LSTM layer.
        x = self.lstm(x)
        # Apply output layer.
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build LSTM model.
lstm_net = LSTM()

In [11]:

# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [12]:
# optimization process
def run_optimization(x, y):
    with tf.GradientTape() as g:
        y_pred = lstm_net(x, is_training=True)
        loss = cross_entropy_loss(y_pred, y)
    gradients = g.gradient(target=loss, sources=lstm_net.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(gradients, lstm_net.trainable_variables))

In [13]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if not step % display_step:
        pred = lstm_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print('step:{} loss:{} accuracy:{}'.format(step, loss, acc))

step:100 loss:0.6311988830566406 accuracy:0.75
step:200 loss:0.5529564619064331 accuracy:0.828125
step:300 loss:0.5148035287857056 accuracy:0.859375
step:400 loss:0.48582643270492554 accuracy:0.890625
step:500 loss:0.495609313249588 accuracy:0.828125
step:600 loss:0.44513675570487976 accuracy:0.875
step:700 loss:0.41907429695129395 accuracy:0.90625
step:800 loss:0.40227368474006653 accuracy:0.921875
step:900 loss:0.3678722083568573 accuracy:0.984375
step:1000 loss:0.39085352420806885 accuracy:0.9375
step:1100 loss:0.34999361634254456 accuracy:1.0
step:1200 loss:0.3725008964538574 accuracy:0.984375
step:1300 loss:0.3522368371486664 accuracy:0.984375
step:1400 loss:0.3735697269439697 accuracy:0.953125
step:1500 loss:0.3887955844402313 accuracy:0.9375
step:1600 loss:0.40360236167907715 accuracy:0.953125
step:1700 loss:0.3445083200931549 accuracy:1.0
step:1800 loss:0.3618878126144409 accuracy:0.96875
step:1900 loss:0.3521907925605774 accuracy:0.984375
step:2000 loss:0.36424171924591064 acc

In [26]:
for step, (x_test, y_test) in enumerate(train_data.take(2), 1):

    # Test model on validation set.
    pred = lstm_net(x_test)
    print("Test Accuracy: %f" % accuracy(pred, y_test))

<class 'tensorflow.python.framework.ops.EagerTensor'>
(64, 20)
